# 랩 3: Amazon AgentCore Identity로 에이전트 생성

## 개요

이 랩에서는 Amazon Bedrock AgentCore 아이덴티티 기능을 통합하여 기존 고객 지원 에이전트를 향상시킬 것입니다. 이를 통해 에이전트가 AgentCore의 아이덴티티 제공자를 통해 적절한 자격 증명 관리를 유지하면서 OAuth2 플로우를 사용하여 Google Calendar과 같은 외부 서비스와 안전하게 인증할 수 있습니다.

캘린더 통합을 통해 지원 에이전트가 고객 대화 내에서 직접 제품 데모나 기술 지원 약속과 같은 이벤트를 예약할 수 있어 지원 워크플로우를 간소화하고 전반적인 고객 경험을 향상시킬 수 있습니다.

![에이전트 아키텍처](images/architecture_lab6_identity.png)

**Based on**: [Official Customer Support Assistant](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)


### 랩 세부 사항

| 정보        | 세부 사항                                                                          |
| :----------------- | :------------------------------------------------------------------------------- |
| 랩 유형           | 점진적 향상                                                                   |
| 에이전트 유형         | 단일                                                                           |
| 에이전트 프레임워크  | Strands Agents                                                                   |
| LLM 모델          | Anthropic Claude Sonnet 4                                                        |
| 랩 구성 요소     | AgentCore 아이덴티티, OAuth2 제공자, Google Calendar API, Cognito 통합  |
| 랩 분야       | 고객 지원                                                                 |
| 예제 복잡도 | 보통                                                                     |
| 사용된 SDK           | Amazon BedrockAgentCore Python SDK, Strands Agents, Google API 클라이언트          |

### 랩 아키텍처

이 랩에서는 아이덴티티 관리 기능으로 고객 지원 에이전트를 확장할 것입니다. 에이전트는 AgentCore 아이덴티티 제공자를 통해 사용자를 인증하고 인증된 사용자를 대신하여 Google Calendar과 같은 외부 서비스에 액세스합니다.

에이전트는 다음을 통합합니다:
- **AgentCore 아이덴티티**: 보안 자격 증명 관리 및 OAuth2 플로우
- **Google OAuth2 제공자**: Google 서비스와의 인증
- **캘린더 도구**: 이벤트 생성 및 캘린더 정보 검색
- **Cognito 제공자** (선택 사항): 사용자 정의 아이덴티티 제공자 통합


### 랩 주요 기능

- 보안 OAuth2 인증 플로우
- 적절한 자격 증명 관리를 통한 외부 서비스 통합
- Google Calendar API 통합
- 사용자 정의 아이덴티티 제공자 구성

## 전제조건

이 튜토리얼을 실행하려면 다음이 필요합니다:

- Python 3.10+
- 구성된 AWS 자격 증명
- Amazon Bedrock AgentCore SDK
- Strands Agents
- **이전에 완료한 랩 1 & 2** - 이 랩은 기존 에이전트를 직접 기반으로 구축됩니다
- **Google Developer Console 액세스** - OAuth2 자격 증명 생성용
- **AgentCore 아이덴티티 권한** - AgentCore 아이덴티티 액세스 권한을 가진 IAM 역할

**참고**: 진행하기 전에 랩 1 & 2의 기본 에이전트가 올바르게 작동하는지 확인하세요.


## 1단계: 종속성 설치 및 라이브러리 가져오기

AgentCore 아이덴티티 통합, Google API 액세스, OAuth2 인증 플로우에 필요한 패키지를 설치하겠습니다. 또한 나중에 사용할 헬퍼 함수도 만들겠습니다.

In [ ]:
# Install required packages
%pip install strands-agents strands-agents-tools "boto3>=1.39.15" python-dotenv utils google-auth google-api-python-client ddgs -q

In [ ]:
# Import libraries
import boto3
import click
import sys
import json
import os
from botocore.exceptions import ClientError

from bedrock_agentcore.identity.auth import requires_access_token
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime, timedelta
from strands import tool
from strands import Agent
from strands.models import BedrockModel
from strands_tools import calculator
import webbrowser
import sys
import os

from lab_helpers.utils import get_ssm_parameter, put_ssm_parameter

session = boto3.session.Session()
region = session.region_name

identity_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=region,
)
cognito = boto3.client('cognito-idp')
ssm = boto3.client("ssm", region_name=region)

print("✅ Libraries imported successfully!")

In [ ]:
# Helper functions to save, retrieve, and delete provider names from SSM

def store_provider_name_in_ssm(provider_name: str):
    """Store credential provider name in SSM parameter."""
    param_name = "/app/customersupport/agentcore/google_provider"
    try:
        ssm.put_parameter(
            Name=param_name, Value=provider_name, Type="String", Overwrite=True
        )
        click.echo(f"🔐 Stored provider name in SSM: {param_name}")
    except ClientError as e:
        click.echo(f"⚠️ Failed to store provider name in SSM: {e}")


def get_provider_name_from_ssm() -> str:
    """Get credential provider name from SSM parameter."""
    param_name = "/app/customersupport/agentcore/google_provider"
    try:
        response = ssm.get_parameter(Name=param_name)
        return response["Parameter"]["Value"]
    except ClientError:
        return None


def delete_ssm_param():
    """Delete SSM parameter for provider."""
    param_name = "/app/customersupport/agentcore/google_provider"
    try:
        ssm.delete_parameter(Name=param_name)
        click.echo(f"🧹 Deleted SSM parameter: {param_name}")
    except ClientError as e:
        click.echo(f"⚠️ Failed to delete SSM parameter: {e}")

## 2단계: 고객 지원 도구 가져오기

에이전트의 핵심 기능을 유지하기 위해 랩 1의 고객 지원 도구를 재사용하겠습니다.


In [ ]:
from lab_helpers.lab1_strands_agent import (
    get_return_policy,
    get_product_info,
)

## 3단계: 메모리 도구 가져오기

에이전트의 핵심 기능을 유지하기 위해 랩 2의 메모리 도구를 재사용하겠습니다.



In [ ]:
from lab_helpers.lab2_helper import setup_memory

memory_hook = setup_memory()

## 4단계: AgentCore 아이덴티티 클라이언트 구성

이 단계에서는 Google Calendar API 기능을 고객 지원 에이전트에 통합하겠습니다. 사용자를 대신하여 Google Calendar과 같은 외부 서비스에 안전하게 액세스하려면 적절한 인증 메커니즘을 구현해야 합니다.

Amazon Bedrock AgentCore Identity는 수동 토큰 관리, 새로고침 처리, 자격 증명 저장의 복잡성을 제거하여 OAuth2 인증 플로우를 관리하는 간소화된 접근 방식을 제공합니다. 이 서비스는 에이전트와 외부 서비스 제공자 간의 보안 중간자 역할을 합니다.

Google OAuth 클라이언트를 생성하려면 아래 단계를 따르세요:

#### ✅ 1. Google Developer Console에서 프로젝트 생성

1. [Google Developer Console](https://console.developers.google.com/)로 이동합니다.
2. 상단 네비게이션 바에서 “프로젝트 생성”을 클릭합니다.
3. 프로젝트 이름을 입력합니다.
4. 조직을 선택하거나 해당 사항이 없으면 “조직 없음”으로 둡니다.
5. 생성을 클릭합니다.

새 프로젝트가 프로젝트 목록에 나타납니다.

#### 📦 2. Google Calendar API 활성화

1. 프로젝트를 선택한 상태에서 왼쪽 메뉴를 열고 APIs & Services > Library로 이동합니다.
2. 검색창에 Google Calendar API를 입력합니다.
3. 결과에서 Google Calendar API를 클릭합니다.
4. 활성화를 클릭합니다.

#### 🛡️ 3. OAuth 동의 화면 구성

1. 왼쪽 메뉴에서 APIs & Services > OAuth consent screen으로 이동합니다.

2. “시작하기”를 클릭합니다.

3. 필수 필드를 입력합니다: 앱 이름, 사용자 지원 이메일
4. 다음을 클릭한 후: 사용자 유형(내부 또는 외부)을 선택합니다. 외부를 선택하는 경우 테스터 이메일 주소를 추가합니다. 개발자 연락처 정보(이메일)를 제공합니다.
5. 약관에 동의하고 완료를 클릭합니다.
6. 생성을 클릭하여 동의 화면을 완료합니다.

#### 🔧 4. OAuth 2.0 자격 증명 생성

1. 왼쪽 메뉴에서 APIs & Services > Credentials로 이동합니다.
2. 자격 증명 생성 > OAuth 클라이언트 ID를 클릭합니다.
3. 애플리케이션 유형으로 웹 애플리케이션을 선택합니다.
4. 자격 증명의 이름을 입력합니다.
5. 승인된 리디렉션 URI 아래에 다음 리디렉션 URI를 추가합니다:
   - `https://bedrock-agentcore.us-east-1.amazonaws.com/identities/oauth2/callback`
6. 생성을 클릭합니다.

#### 🔑 5. 클라이언트 ID 및 클라이언트 시크릿 획득

생성 후 대화상자에 클라이언트 ID와 클라이언트 시크릿이 표시됩니다. JSON 다운로드를 클릭하여 자격 증명을 파일로 저장합니다. 이 파일을 프로젝트의 `credentials.json`으로 저장합니다. (파일 이름을 맞추기 위해 이름을 변경해야 할 수도 있습니다)

#### 🔍 6. 데이터 액세스 범위 업데이트

1. APIs & Services > Credentials로 이동합니다.
2. 생성한 OAuth 2.0 클라이언트 ID를 클릭합니다.
3. 왼쪽 메뉴에서 Data access를 선택합니다.
4. “범위 추가 또는 제거”를 클릭합니다.
5. 수동으로 범위 추가 아래에 범위를 입력합니다: `https://www.googleapis.com/auth/calendar`
6. 업데이트를 클릭한 다음 저장을 클릭하여 구성을 확인합니다.

In [ ]:
credentials_file = "credentials.json"

#Verify credentials file looks as expected, and extract credentials

if not os.path.isfile(credentials_file):
    print(f"❌ Error: '{credentials_file}' file not found")
    sys.exit(1)

print(f"📄 Reading credentials from {credentials_file}...")
try:
    with open(credentials_file, "r") as f:
        data = json.load(f)
except json.JSONDecodeError as e:
    print(f"❌ Error parsing JSON: {e}")
    sys.exit(1)

web_config = data.get("web")
if not web_config:
    print("❌ Error: 'web' section missing in credentials.json")
    sys.exit(1)

client_id = web_config.get("client_id")
client_secret = web_config.get("client_secret")

if not client_id:
    print("❌ Error: 'client_id' not found in credentials.json")
    sys.exit(1)

if not client_secret:
    print("❌ Error: 'client_secret' not found in credentials.json")
    sys.exit(1)

print("✅ Client ID and Secret loaded from credentials.json")

In [ ]:
google_provider_name = "customersupport-google-calendar"

try:
    print("🔧 Creating Google OAuth2 credential provider...")
    google_provider = identity_client.create_oauth2_credential_provider(
        name=google_provider_name,
        credentialProviderVendor="GoogleOauth2",
        oauth2ProviderConfigInput={
            "googleOauth2ProviderConfig": {
                "clientId": client_id,
                "clientSecret": client_secret,
            }
        },
    )

    print("✅ Google OAuth2 credential provider created successfully")
    google_provider_arn = google_provider["credentialProviderArn"]
    print(f"   Provider ARN: {google_provider_arn}")
    print(f"   Provider Name: {google_provider['name']}")

    # Store provider name in SSM
    store_provider_name_in_ssm(google_provider_name)
except Exception as e:
    print(f"❌ Error creating Google credential provider: {str(e)}")

In [ ]:
# List all OAuth2 credential providers.
try:
    response = identity_client.list_oauth2_credential_providers(maxResults=20)
    providers = response.get("credentialProviders", [])
    print(providers)
except Exception as e:
    print(f"❌ Error listing credential providers: {str(e)}", err=True)

이제 AgentCore Identity를 사용하여 Google Calendar로 인증하고 사용자를 대신하여 캘린더 작업을 수행하는 도구를 만들겠습니다.

먼저 OAuth2 인증 플로우를 구성해보겠습니다:

In [ ]:
async def on_auth_url(url: str):
    webbrowser.open(url)
    
SCOPES = ["https://www.googleapis.com/auth/calendar"]

google_access_token = None

@requires_access_token(
    provider_name=google_provider_name,
    scopes=["https://www.googleapis.com/auth/calendar"],  # Google OAuth2 scopes
    auth_flow="USER_FEDERATION",  # On-behalf-of user (3LO) flow
    on_auth_url=on_auth_url,  # prints authorization URL to console
    force_authentication=True,
    into="access_token",
)

def get_google_access_token(access_token: str):
    return access_token

이제 캘린더 관리 도구를 만들어보겠습니다:

In [ ]:
@tool(
    name="Create_calendar_event",
    description="Creates a new event on your Google Calendar",
)
def create_calendar_event() -> str:
    google_access_token = ''
    try:
        google_access_token = get_google_access_token(access_token=google_access_token)
        if not google_access_token:
            raise Exception("requires_access_token did not provide tokens")
    except Exception as e:
        return "Error Authentication with Google: " + str(e)

    creds = Credentials(token=google_access_token, scopes=SCOPES)

    try:
        service = build("calendar", "v3", credentials=creds)

        # Define event details
        start_time = datetime.now() + timedelta(hours=1)
        end_time = start_time + timedelta(hours=1)

        event = {
            "summary": "Test Event from API",
            "location": "Virtual",
            "description": "This event was created using the Google Calendar API.",
            "start": {
                "dateTime": start_time.isoformat() + "Z",  # UTC time
                "timeZone": "UTC",
            },
            "end": {
                "dateTime": end_time.isoformat() + "Z",
                "timeZone": "UTC",
            },
        }

        created_event = (
            service.events().insert(calendarId="primary", body=event).execute()
        )

        return json.dumps(
            {
                "event_created": True,
                "event_id": created_event.get("id"),
                "htmlLink": created_event.get("htmlLink"),
            }
        )

    except HttpError as error:
        return json.dumps({"error": str(error), "event_created": False})
    except Exception as e:
        return json.dumps({"error": str(e), "event_created": False})


In [ ]:
@tool(
    name="Get_calendar_events_today",
    description="Retrieves the calendar events for the day from your Google Calendar",
)
def get_calendar_events_today() -> str:
    google_access_token = ''
    try:
        google_access_token = get_google_access_token(
            access_token=google_access_token)

        if not google_access_token:
            raise Exception("requires_access_token did not provide tokens")
    except Exception as e:
        return "Error Authentication with Google: " + str(e)

    # Create credentials from the provided access token
    creds = Credentials(token=google_access_token, scopes=SCOPES)
    try:
        service = build("calendar", "v3", credentials=creds)
        # Call the Calendar API
        today_start = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        today_end = today_start.replace(hour=23, minute=59, second=59)

        # Format with CDT timezone (-05:00)
        timeMin = today_start.strftime("%Y-%m-%dT00:00:00-05:00")
        timeMax = today_end.strftime("%Y-%m-%dT23:59:59-05:00")

        events_result = (
            service.events()
            .list(
                calendarId="primary",
                timeMin=timeMin,
                timeMax=timeMax,
                singleEvents=True,
                orderBy="startTime",
            )
            .execute()
        )
        events = events_result.get("items", [])
        if not events:
            return json.dumps({"events": []})  # Return empty events array as JSON

        return json.dumps({"events": events})  # Return events wrapped in an object
    except HttpError as error:
        error_message = str(error)
        return json.dumps({"error": error_message, "events": []})
    except Exception as e:
        error_message = str(e)
        return json.dumps({"error": error_message, "events": []})

새로운 캘린더 기능을 갖춘 고객 지원 에이전트를 만들어보겠습니다:

In [ ]:
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
system_prompt = """
    You are a helpful and professional customer support assistant for an electronics e-commerce company.
Your role is to:
- Provide accurate information using the tools available to you
- Support the customer with technical information and product specifications.
- Be friendly, patient, and understanding with customers
- Always offer additional help after answering questions
- If you can't help with something, direct customers to the appropriate contact

You have access to the following tools:
1. get_return_policy() - For warranty and return policy questions
2. get_product_info() - To get information about a specific product
3. web_search() - To access current technical documentation, or for updated information. 
4. create_calendar_event() - To create a new calendar event
5. get_calendar_events_today() - To find events on the calendar today
Always use the appropriate tool to get accurate, up-to-date information rather than making assumptions about electronic products or specifications.
    """
agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=[create_calendar_event, get_calendar_events_today],
    callback_handler=None,
)

이제 에이전트를 테스트할 시간입니다! ```인증 URL에 대한 토큰 폴링 중```이라는 메시지와 함께 URL이 표시됩니다. 이 URL을 클릭하여 Google 계정에 로그인하고 에이전트가 Google 캘린더에 액세스할 수 있는 권한을 부여하세요.

In [ ]:
print(str(agent(
            "Can you create a new event on my cal? You can call the create_calendar_event directly."
        )))


In [ ]:
print(str(agent("Whats my agenda for today?")))

## 축하합니다! 🎉

**Amazon AgentCore 아이덴티티 기능으로 에이전트 생성**을 성공적으로 완료했습니다!

### 달성한 내용:

✅ **AgentCore 아이덴티티 구성**: 보안 인증을 위한 OAuth2 자격 증명 제공자 설정
✅ **Google Calendar 통합**: 캘린더 이벤트 관리 및 보기를 위한 도구 생성
✅ **고객 지원 향상**: 일정 및 캘린더 기능으로 에이전트 확장

### 주요 학습 내용:
- **아이덴티티 관리**: 보안 자격 증명 관리를 위한 AgentCore 아이덴티티 사용
- **OAuth2 플로우**: 사용자 연동 및 토큰 기반 인증 구현
- **외부 API 통합**: Google Calendar과 같은 타사 서비스에 안전하게 연결
- **도구 향상**: 기존 에이전트 도구에 아이덴티티 인식 기능 추가

## 다음 단계

에이전트를 더욱 향상시킬 준비가 되셨나요? 다음으로 계속하세요:

- **랩 4**: Gateway를 활용하여 도구와 기타 리소스를 안전하게 연결
- **랩 5**: 프로덕션 모니터링을 위한 관찰 가능성 및 가드레일 구현
- **랩 6**: 확장 가능한 프로덕션 호스팅을 위해 AgentCore Runtime에 배포

## 리소스

- [AgentCore 아이덴티티 문서](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore-아이덴티티.html)
- [Google Calendar API 문서](https://developers.google.com/calendar/API)
- [Strands Agents 문서](https://github.com/strands-agents/sdk-python)
- [Amazon Bedrock 모델](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html)
- [공식 고객 지원 샘플](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)

---

**훌륭한 작업입니다! 고객 지원 에이전트가 이제 보안 아이덴티티 관리와 캘린더 통합 기능을 갖게 되었습니다! 🚀**

## (선택 사항): Cognito로 아이덴티티 제공자 생성

추가 아이덴티티 관리 기능을 위해 Cognito 기반 아이덴티티 제공자도 구성할 수 있습니다. 이 섹션에서는 Amazon Cognito를 사용하여 사용자 정의 OAuth2 제공자를 만드는 방법을 보여줍니다:


In [ ]:
cognito_provider_name = "customersupport-gateways-cognito"

try:
    print("📥 Fetching Cognito configuration from SSM...")
    
    client_id = get_ssm_parameter("/app/customersupport/agentcore/machine_client_id")
    print(f"✅ Retrieved client ID: {client_id}")

    client_secret = get_ssm_parameter("/app/customersupport/agentcore/cognito_secret")
    print(f"✅ Retrieved client secret: {client_secret[:4]}***")

    issuer = get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
    auth_url = get_ssm_parameter("/app/customersupport/agentcore/cognito_auth_url")
    token_url = get_ssm_parameter("/app/customersupport/agentcore/cognito_token_url")

    print(f"✅ Issuer: {issuer}")
    print(f"✅ Authorization Endpoint: {auth_url}")
    print(f"✅ Token Endpoint: {token_url}")

    print("⚙️  Creating OAuth2 credential provider...")
    
    cognito_provider = identity_client.create_oauth2_credential_provider(
            name=cognito_provider_name,
            credentialProviderVendor="CustomOauth2",
            oauth2ProviderConfigInput={
                "customOauth2ProviderConfig": {
                    "clientId": client_id,
                    "clientSecret": client_secret,
                    "oauthDiscovery": {
                        "authorizationServerMetadata": {
                            "issuer": issuer,
                            "authorizationEndpoint": auth_url,
                            "tokenEndpoint": token_url,
                            "responseTypes": ["code", "token"],
                        }
                    },
                }
            },
        )

    provider_arn = cognito_provider["credentialProviderArn"]
    print(provider_arn)
except Exception as e:
        print(f"❌ Error creating Cognito credential provider: {str(e)}")

In [ ]:
response = identity_client.list_oauth2_credential_providers(maxResults=20)
providers = response.get("credentialProviders", [])
print(providers)

## 정리

In [ ]:
#  try:
#     print(f"🗑️  Deleting Google OAuth2 credential provider: {google_provider_name}")
#     identity_client.delete_oauth2_credential_provider(name=google_provider_name)
#     print("✅ Google OAuth2 credential provider deleted successfully")
# except Exception as e:
#     print(f"❌ Error deleting credential provider: {str(e)}")

In [ ]:
#  try:
#     print(f"🗑️  Deleting Cognito OAuth2 credential provider: {cognito_provider_name}")
#     identity_client.delete_oauth2_credential_provider(name=cognito_provider_name)
#     print("✅ Cognito credential provider deleted successfully")
# except Exception as e:
#     print(f"❌ Error deleting credential provider: {str(e)}")